# Information Retriever: Building Blocks

## Env set up

In [0]:
!python3 --version

In [0]:
%pip install -U -qqqq --upgrade databricks-sdk
dbutils.library.restartPython()

## Example data

In [0]:
## Creating a dataframe with examples of clients data
columns = ["customer_id", "name", "surname", "email", "address"]
examples_ = [
    (1, "Lena", "Schmidt", "lena.schmidt@example.com",
     "Alexanderplatz 5, 10178 Berlin, Germany"),
    (2, "Jonas", "Müller", "jonas.mueller@example.com",
     "221B Baker Street, NW1 6XE London, United Kingdom"),
    (3, "Sofia", "Rossi", "sofia.rossi@example.com",
     "Via Milano 11, 20126 Milano, Italy"),
    (4, "Marek", "Nowak", "marek.nowak@example.com",
     "ul. Marszałkowska 10, 00-590 Warsaw, Poland"),
    (5, "Hannah", "Dubois", "hannah.dubois@example.com",
     "12 Rue de la Paix, 75002 Paris, France"),
    (6, "David", "Nguyen", "david.nguyen@example.com",
     "Damrak 45, 1012 LL Amsterdam, Netherlands"),
    (7, "Amira", "Hassan", "amira.hassan@example.com",
     "Gran Vía 28, 28013 Madrid, Spain"),
    (8, "Felix", "Kovac", "felix.kovac@example.com",
     "Ringstrasse 3, 1010 Vienna, Austria"),
    (9, "Carla", "Lindberg", "carla.lindberg@example.com",
     "Sveavägen 15, 111 57 Stockholm, Sweden"),
    (10, "Tobias", "Nielsen", "tobias.nielsen@example.com",
     "Rådhuspladsen 1, 1550 Copenhagen, Denmark")
]

df_customer_example = spark.createDataFrame(examples_, columns)
display(df_customer_example)

In [0]:
## Creating a dataframe with examples of order data
order_columns = ["Order_ID", "date", "cart", "amount", "customer_id"]
examples_ = [
    ("ID-45892171", "2025-11-02",
     ["Office Chair", "Standing Desk", "Desk Lamp"],
     1299.90, 1),

    ("ID-45892172", "2025-11-03",
     ["Mechanical Keyboard", "Wireless Mouse", "Mouse Pad"],
     189.50, 2),

    ("ID-45892173", "2025-11-05",
     ["A4 Paper (500 sheets)", "Stapler", "Staples Pack", "Highlighters Set"],
     64.30, 3),

    ("ID-45892174", "2025-11-06",
     ["Whiteboard", "Whiteboard Markers Set", "Eraser"],
     215.00, 4),

    ("ID-45892175", "2025-11-08",
     ["Desk Organizer", "Notebooks Pack", "Ballpoint Pens (20x)"],
     72.40, 5),

    ("ID-45892176", "2025-11-09",
     ["Monitor 27\"", "HDMI Cable", "Monitor Arm"],
     459.99, 6),

    ("ID-45892177", "2025-11-10",
     ["Office Chair", "Footrest"],
     389.00, 7),

    ("ID-45892178", "2025-11-11",
     ["Laser Printer", "Printer Paper (1000 sheets)"],
     329.50, 8),

    ("ID-45892179", "2025-11-12",
     ["Inkjet Printer Cartridges Set", "Label Printer"],
     248.75, 9),

    ("ID-45892180", "2025-11-13",
     ["Conference Speakerphone", "Webcam HD"],
     312.20, 10),

    ("ID-45892181", "2025-11-14",
     ["Filing Cabinet", "Hanging Folders (25x)"],
     410.00, 1),

    ("ID-45892182", "2025-11-15",
     ["Desk Lamp", "LED Bulbs (4x)", "Cable Management Kit"],
     98.60, 2),

    ("ID-45892183", "2025-11-16",
     ["Mouse Pad", "Ergonomic Wrist Rest"],
     42.30, 3),

    ("ID-45892184", "2025-11-18",
     ["A4 Paper (500 sheets)", "Sticky Notes Pack", "Markers Set"],
     53.10, 4),

    ("ID-45892185", "2025-11-19",
     ["Flipchart", "Flipchart Paper (3x)", "Markers Set"],
     187.40, 5),

    ("ID-45892186", "2025-11-20",
     ["Network Switch 8-port", "Ethernet Cables (5x)"],
     224.80, 6),

    ("ID-45892187", "2025-11-21",
     ["External SSD 1TB", "USB Hub"],
     189.99, 7),

    ("ID-45892188", "2025-11-22",
     ["Office Headset", "Laptop Stand"],
     154.60, 8),

    ("ID-45892189", "2025-11-23",
     ["Desk Organizer", "Pen Holder", "Document Tray"],
     61.25, 9),

    ("ID-45892190", "2025-11-24",
     ["Monitor 24\"", "DisplayPort Cable"],
     269.90, 10),

    ("ID-45892191", "2025-11-25",
     ["Whiteboard Markers Set", "Cleaning Spray", "Microfiber Cloths"],
     39.80, 1),

    ("ID-45892192", "2025-11-26",
     ["Notebooks Pack", "Ballpoint Pens (50x)", "Highlighters Set"],
     88.15, 2),

    ("ID-45892193", "2025-11-27",
     ["Office Chair", "Seat Cushion"],
     345.00, 3),

    ("ID-45892194", "2025-11-28",
     ["Paper Shredder", "Trash Bags (Office)"],
     219.50, 4),

    ("ID-45892195", "2025-11-29",
     ["Laminator", "Laminating Pouches (100x)"],
     132.40, 5)
]

df_order_example = spark.createDataFrame(examples_, order_columns)
display(df_order_example)

## Persist Data
Execute this section only when the data needs to be recreated.

Sync delta tables to Lakebase ([doc](https://docs.databricks.com/aws/en/oltp/instances/sync-data/sync-table?language=Python+SDK))

In [0]:
regenerate_data = False

In [0]:
## Variables containing the pointers to catalog, schema, lakebase

catalog_ = 'users'
schema_ = 'gabriele_albini'
lakebase_instance_ = 'shared-instance-size-4'

In [0]:
## Generate delta tables
if regenerate_data:
  spark.sql("CREATE CATALOG IF NOT EXISTS "+catalog_)
  spark.sql("USE CATALOG "+catalog_)
  spark.sql("CREATE SCHEMA IF NOT EXISTS "+schema_)
  spark.sql("USE SCHEMA "+schema_)
  spark.sql("DROP TABLE IF EXISTS classificator_agent_customers")
  spark.sql("DROP TABLE IF EXISTS classificator_agent_orders")
  df_customer_example.write.mode("overwrite").saveAsTable("classificator_agent_customers")
  spark.sql(
  """
    ALTER TABLE classificator_agent_customers ALTER COLUMN customer_id SET NOT NULL
  """)
  spark.sql(
  """
    ALTER TABLE classificator_agent_customers ADD CONSTRAINT customers_pk PRIMARY KEY (customer_id)
  """)
  df_order_example.write.mode("overwrite").saveAsTable("classificator_agent_orders")
  spark.sql(
  """
    ALTER TABLE classificator_agent_orders ALTER COLUMN Order_ID SET NOT NULL
  """)
  spark.sql(
  """
    ALTER TABLE classificator_agent_orders ADD CONSTRAINT orders_pk PRIMARY KEY (Order_ID)
  """)

else:
  spark.sql("USE CATALOG "+catalog_)
  spark.sql("USE SCHEMA "+schema_)

In [0]:
## Sync Delta tables with Lakebase Instance: Customers

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance
from databricks.sdk.service.database import SyncedDatabaseTable, SyncedTableSpec, NewPipelineSpec, SyncedTableSchedulingPolicy
w = WorkspaceClient()

if regenerate_data:    

    # Create a synced table in a standard UC catalog
    synced_table = w.database.create_synced_database_table(
        SyncedDatabaseTable(
            name=catalog_+"."+schema_+".classificator_agent_customers_synced",  # Full three-part name
            database_instance_name=lakebase_instance_,  # Required for standard catalogs
            logical_database_name="databricks_postgres",  # Required for standard catalogs
            spec=SyncedTableSpec(
                source_table_full_name=catalog_+"."+schema_+".classificator_agent_customers",
                primary_key_columns=["customer_id"],
                scheduling_policy=SyncedTableSchedulingPolicy.SNAPSHOT,
                create_database_objects_if_missing=True,  # Create database/schema if needed
                new_pipeline_spec=NewPipelineSpec(
                    storage_catalog=catalog_,
                    storage_schema=schema_
                )
            ),
        )
    )
    print(f"Created synced table: {synced_table.name}")

    synced_table_name = catalog_+"."+schema_+".classificator_agent_customers_synced"
    status = w.database.get_synced_database_table(name=synced_table_name)
    print(f"Synced table status: {status.data_synchronization_status.detailed_state}")
    print(f"Status message: {status.data_synchronization_status.message}")

# Check the status of a synced table
synced_table_name = catalog_+"."+schema_+".classificator_agent_customers_synced"
status = w.database.get_synced_database_table(name=synced_table_name)
print(status)

In [0]:
## Sync Delta tables with Lakebase Instance: Orders

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance
from databricks.sdk.service.database import SyncedDatabaseTable, SyncedTableSpec, NewPipelineSpec, SyncedTableSchedulingPolicy
w = WorkspaceClient()

if regenerate_data:    

    # Create a synced table in a standard UC catalog
    synced_table = w.database.create_synced_database_table(
        SyncedDatabaseTable(
            name=catalog_+"."+schema_+".classificator_agent_orders_synced",  # Full three-part name
            database_instance_name=lakebase_instance_,  # Required for standard catalogs
            logical_database_name="databricks_postgres",  # Required for standard catalogs
            spec=SyncedTableSpec(
                source_table_full_name=catalog_+"."+schema_+".classificator_agent_orders",
                primary_key_columns=["Order_ID"],
                scheduling_policy=SyncedTableSchedulingPolicy.SNAPSHOT,
                create_database_objects_if_missing=True,  # Create database/schema if needed
                new_pipeline_spec=NewPipelineSpec(
                    storage_catalog=catalog_,
                    storage_schema=schema_
                )
            ),
        )
    )
    print(f"Created synced table: {synced_table.name}")

    synced_table_name = catalog_+"."+schema_+".classificator_agent_orders_synced"
    status = w.database.get_synced_database_table(name=synced_table_name)
    print(f"Synced table status: {status.data_synchronization_status.detailed_state}")
    print(f"Status message: {status.data_synchronization_status.message}")

# Check the status of a synced table
synced_table_name = catalog_+"."+schema_+".classificator_agent_orders_synced"
status = w.database.get_synced_database_table(name=synced_table_name)
print(status)

## Create Retriever

Create a UC function that can be used as a retriever tool by the agent ([doc](https://docs.databricks.com/aws/en/generative-ai/agent-framework/structured-retrieval-tools), Databricks Demos [example in Notebook 5.1](https://notebooks.databricks.com/demos/lakehouse-iot-platform/index.html#))

In [0]:
%sql
CREATE OR REPLACE FUNCTION turbine_specifications_retriever(turbine_id STRING)
RETURNS STRUCT<turbine_id STRING, hourly_timestamp STRING, avg_energy DOUBLE, std_sensor_A DOUBLE, std_sensor_B DOUBLE, std_sensor_C DOUBLE, std_sensor_D DOUBLE, std_sensor_E DOUBLE, std_sensor_F DOUBLE, country STRING, lat STRING, location STRING, long STRING, model STRING, state STRING>
LANGUAGE SQL
COMMENT 'Returns the specifications of one specific turbine based on its turbine id'
RETURN (
  SELECT struct(turbine_id, hourly_timestamp, avg_energy, std_sensor_A, std_sensor_B, std_sensor_C, std_sensor_D, std_sensor_E, std_sensor_F, country, lat, location, long, model, state)
  FROM turbine_current_features_synced
  WHERE turbine_id = turbine_specifications_retriever.turbine_id
  LIMIT 1
);